In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [22]:
df = pd.read_csv("europe_clean.csv")

In [23]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,date_int,new_cases_smooth_per_pop_den,num_of_days_data_avail
0,ALB,Europe,Albania,2020-03-14,38,5,5.429,1.0,0.0,0.143,...,304.19501,10.08,7.1,51.200001,2.89,78.57,0.795,14mar2020,0.051768,355
1,ALB,Europe,Albania,2020-03-15,42,4,6.000,1.0,0.0,0.143,...,304.19501,10.08,7.1,51.200001,2.89,78.57,0.795,15mar2020,0.057213,355
2,ALB,Europe,Albania,2020-03-16,51,9,7.000,1.0,0.0,0.143,...,304.19501,10.08,7.1,51.200001,2.89,78.57,0.795,16mar2020,0.066749,355
3,ALB,Europe,Albania,2020-03-17,55,4,6.429,1.0,0.0,0.143,...,304.19501,10.08,7.1,51.200001,2.89,78.57,0.795,17mar2020,0.061304,355
4,ALB,Europe,Albania,2020-03-18,59,4,6.714,2.0,1.0,0.143,...,304.19501,10.08,7.1,51.200001,2.89,78.57,0.795,18mar2020,0.064022,355


In [20]:
# df["year"] = df["date"]
# df["month"] = df["date"]
# df["day"] = df["date"]

In [ ]:
# for index, row in df.iterrows():
#     df["year"][index] = df["date"].str.split("-")[index][0]
#     df["month"] = df["date"].str.split("-")[1]
#     df["day"] = df["date"].str.split("-")[2]

In [25]:
def split_date(date):
    year, month, day = date.split('-')

    return pd.Series({
        'year': year,
        'month': month,
        'day': day
    })

# df_new has the new columns
df_new = df['date'].apply(split_date)

# append the columns to the original dataframe
df = pd.concat([df, df_new],axis=1)

In [26]:
df[["year", "month", "day"]]

,year,month,day
0,2020,03,14
1,2020,03,15
2,2020,03,16
3,2020,03,17
4,2020,03,18
...,...,...,...
10818,2021,02,27
10819,2021,02,28
10820,2021,03,01
10821,2021,03,02


In [27]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_millio', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_milli', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hundre

In [28]:
df["new_deaths_smoothed_per_million"]

0        0.050
1        0.050
2        0.050
3        0.050
4        0.050
         ...  
10818    4.937
10819    4.783
10820    4.628
10821    4.196
10822    4.289
Name: new_deaths_smoothed_per_million, Length: 10823, dtype: float64

In [29]:
data_X = df[['year', 'month', 'day', 'median_age', 'stringency_index', 'female_smokers', 'male_smokers', 'population_density', 
             'hospital_beds_per_thousand', 'human_development_index']]
data_Y = df[['new_cases_smoothed_per_million']]

In [32]:
data_X.dropna()

,year,month,day,median_age,stringency_index,female_smokers,male_smokers,population_density,hospital_beds_per_thousand,human_development_index
0,2020,03,14,38.000000,78.699997,7.1,51.200001,104.87100,2.89,0.795
1,2020,03,15,38.000000,81.480003,7.1,51.200001,104.87100,2.89,0.795
2,2020,03,16,38.000000,81.480003,7.1,51.200001,104.87100,2.89,0.795
3,2020,03,17,38.000000,81.480003,7.1,51.200001,104.87100,2.89,0.795
4,2020,03,18,38.000000,81.480003,7.1,51.200001,104.87100,2.89,0.795
...,...,...,...,...,...,...,...,...,...,...
10813,2021,02,22,40.799999,87.959999,20.0,24.700001,272.89801,2.54,0.932
10814,2021,02,23,40.799999,87.959999,20.0,24.700001,272.89801,2.54,0.932
10815,2021,02,24,40.799999,87.959999,20.0,24.700001,272.89801,2.54,0.932
10816,2021,02,25,40.799999,87.959999,20.0,24.700001,272.89801,2.54,0.932


In [33]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.3)

In [34]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').